In [ ]:
#| default_exp renderers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch

## Siddon's Method

DRRs are generated by modeling the geometry of an idealized projectional radiography system.
Let $\mathbf s \in \mathbb R^3$ be the X-ray source and $\mathbf p \in \mathbb R^3$ be a target pixel on the detector plane.
Then, $R(\alpha) = \mathbf s + \alpha (\mathbf p - \mathbf s)$ is a ray that originates from $\mathbf s$ ($\alpha=0$), passes through the imaged volume, and hits the detector plane at $\mathbf p$ ($\alpha=1$).
The proportion of energy attenuation experienced by the X-ray at the time it reaches pixel $\mathbf p$ is given by the following line integral:

\begin{equation}
    E(R) = \|\mathbf p - \mathbf s\|_2 \int_0^1 \mathbf V \left( \mathbf s + \alpha (\mathbf p - \mathbf s) \right) \, \mathrm d\alpha \,,
\end{equation}

where $\mathbf V : \mathbb R^3 \mapsto \mathbb R$ is the imaged volume.
The units term $\|\mathbf p - \mathbf s\|_2$ serves to cancel out the units of $\mathbf V(\cdot)$, reciprocal length, such that the final proportion $E$ is unitless.
For DRR synthesis, $\mathbf V$ is approximated by a discrete 3D CT volume, and the first equation becomes

\begin{equation}
    E(R) = \|\mathbf p - \mathbf s\|_2 \sum_{m=1}^{M-1} (\alpha_{m+1} - \alpha_m) \mathbf V \left[ \mathbf s + \frac{\alpha_{m+1} + \alpha_m}{2} (\mathbf p - \mathbf s) \right] \,,
\end{equation}

where $\alpha_m$ parameterizes the locations where ray $R$ intersects one of the orthogonal planes comprising the CT volume, and $M$ is the number of such intersections.

Siddon's method provides a parametric method to identify the plane intersections $\{\alpha_m\}_{m=1}^M$.
Let $\Delta X$ be the CT voxel size in the $x$-direction and $b_x$ be the location of the $0$-th plane in this direction.
Then the intersection of ray $R$ with the $i$-th plane in the $x$-direction is given by
\begin{equation}
    \alpha_x(i) = \frac{b_x + i \Delta X - \mathbf s_x}{\mathbf p_x - \mathbf s_x} ,
\end{equation}
with analogous expressions for $\alpha_y(\cdot)$ and $\alpha_z(\cdot)$.

We can use this equation to compute the values $\mathbf \alpha_x$ for all the intersections between $R$ and the planes in the $x$-direction:
\begin{equation}
    \mathbf\alpha_x = \{ \alpha_x(i_{\min}), \dots, \alpha_x(i_{\max}) \} ,
\end{equation}
where $i_{\min}$ and $i_{\max}$ denote the first and last intersections of $R$ with the $x$-direction planes.

Defining $\mathbf\alpha_y$ and $\mathbf\alpha_z$ analogously, we construct the array
\begin{equation}
    \mathbf\alpha = \mathrm{sort}(\mathbf\alpha_x, \mathbf\alpha_y, \mathbf\alpha_z) ,
\end{equation}
which contains $M$ values of $\alpha$ parameterizing the intersections between $R$ and the orthogonal $x$-, $y$-, and $z$-directional planes. 
We substitute values in the sorted set $\mathbf\alpha$ into the first equation to evaluate $E(R)$, which corresponds to the intensity of pixel $\mathbf p$ in the synthesized DRR.

In [ ]:
#| export
class Siddon(torch.nn.Module):
    """Differentiable X-ray renderer implemented with Siddon's method for exact raytracing."""
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps

    def dims(self, volume):
        return torch.tensor(volume.shape).to(volume) + 1

    def maxidx(self, volume):
        return volume.numel() - 1

    def forward(self, volume, spacing, source, target):
        dims = self.dims(volume)
        maxidx = self.maxidx(volume)

        alphas = _get_alphas(source, target, spacing, dims, self.eps)
        alphamid = (alphas[..., 0:-1] + alphas[..., 1:]) / 2
        voxels = _get_voxel(
            alphamid, source, target, volume, spacing, dims, maxidx, self.eps
        )

        # Step length for alphas out of range will be nan
        # These nans cancel out voxels convereted to 0 index
        step_length = torch.diff(alphas, dim=-1)
        weighted_voxels = voxels * step_length

        drr = torch.nansum(weighted_voxels, dim=-1)
        raylength = (target - source + self.eps).norm(dim=-1)
        drr *= raylength
        return drr

In [ ]:
#| export
def _get_alphas(source, target, spacing, dims, eps):
    # Get the CT sizing and spacing parameters
    alphax = torch.arange(dims[0]).to(source) * spacing[0]
    alphay = torch.arange(dims[1]).to(source) * spacing[1]
    alphaz = torch.arange(dims[2]).to(source) * spacing[2]

    # Get the alpha at each plane intersection
    sx, sy, sz = source[..., 0], source[..., 1], source[..., 2]
    alphax = alphax.expand(len(source), 1, -1) - sx.unsqueeze(-1)
    alphay = alphay.expand(len(source), 1, -1) - sy.unsqueeze(-1)
    alphaz = alphaz.expand(len(source), 1, -1) - sz.unsqueeze(-1)

    sdd = target - source + eps
    alphax = alphax / sdd[..., 0].unsqueeze(-1)
    alphay = alphay / sdd[..., 1].unsqueeze(-1)
    alphaz = alphaz / sdd[..., 2].unsqueeze(-1)
    alphas = torch.cat([alphax, alphay, alphaz], dim=-1)

    # Get the alphas within the range [alphamin, alphamax]
    alphamin, alphamax = _get_alpha_minmax(source, target, spacing, dims, eps)
    good_idxs = torch.logical_and(alphas >= alphamin, alphas <= alphamax)
    alphas[~good_idxs] = torch.nan

    # Sort the alphas by ray, putting nans at the end of the list
    alphas = torch.sort(alphas, dim=-1).values
    
    # Drop indices where alphas for all rays are nan
    alphas = alphas[..., ~alphas.isnan().all(dim=0).all(dim=0)]

    return alphas


def _get_alpha_minmax(source, target, spacing, dims, eps):
    sdd = target - source + eps
    planes = torch.zeros(3).to(source)
    alpha0 = (planes * spacing - source) / sdd
    planes = (dims - 1).to(source)
    alpha1 = (planes * spacing - source) / sdd
    alphas = torch.stack([alpha0, alpha1]).to(source)

    alphamin = alphas.min(dim=0).values.max(dim=-1).values.unsqueeze(-1)
    alphamax = alphas.max(dim=0).values.min(dim=-1).values.unsqueeze(-1)

    alphamin = torch.where(alphamin < 0.0, 0.0, alphamin)
    alphamax = torch.where(alphamax > 1.0, 1.0, alphamax)
    return alphamin, alphamax


def _get_voxel(alpha, source, target, volume, spacing, dims, maxidx, eps):
    idxs = _get_index(alpha, source, target, spacing, dims, maxidx, eps)
    return torch.take(volume, idxs)


def _get_index(alpha, source, target, spacing, dims, maxidx, eps):
    sdd = target - source + eps
    idxs = source.unsqueeze(1) + alpha.unsqueeze(-1) * sdd.unsqueeze(2)
    idxs = idxs / spacing
    idxs = idxs.trunc()
    # Conversion to long makes nan->-inf, so temporarily replace them with 0
    # This is cancelled out later by multiplication by nan step_length
    idxs = (
        idxs[..., 0] * (dims[1] - 1) * (dims[2] - 1)
        + idxs[..., 1] * (dims[2] - 1)
        + idxs[..., 2]
    ).long() + 1
    idxs[idxs < 0] = 0
    idxs[idxs > maxidx] = maxidx
    return idxs

## Compilier-friendly Siddon

## Trilinear interpolation

Instead of computing the exact line integral over the voxel grid (i.e., Siddon's method), we can sample colors at points along the each ray using trilinear interpolation.

Now, the rendering equation is
\begin{equation}
    E(R) = \|\mathbf p - \mathbf s\|_2\frac{1}{M} \sum_{m=1}^{M} \mathbf V \left[ \mathbf s + \alpha_m (\mathbf p - \mathbf s) \right] \,,
\end{equation}
where $\mathbf V[\cdot]$ is the trilinear interpolation function and $M$ is the number of points sampled per ray.

In [ ]:
#| export
from torch.nn.functional import grid_sample


class Trilinear(torch.nn.Module):
    """Differentiable X-ray renderer implemented with trilinear interpolation."""
    def __init__(
        self,
        near=0.0,
        far=1.0,
        mode="bilinear",
        eps=1e-8,
    ):
        super().__init__()
        self.near = near
        self.far = far
        self.mode = mode
        self.eps = eps

    def dims(self, volume):
        return torch.tensor(volume.shape).to(volume) - 1

    def forward(self, volume, spacing, source, target, n_points=250, align_corners=True):
        # Get the raylength and reshape sources
        raylength = (source - target + self.eps).norm(dim=-1)
        source = source[:, None, :, None, :]
        target = target[:, None, :, None, :]

        # Sample points along the rays and rescale to [-1, 1]
        alphas = torch.linspace(self.near, self.far, n_points).to(volume)
        alphas = alphas[None, None, None, :, None]
        rays = source + alphas * (target - source)
        rays = 2 * rays / (spacing * self.dims(volume)) - 1

        # Reorder array to match torch conventions
        volume = volume.permute(2, 1, 0)
        
        # Render the DRR
        batch_size = len(rays)
        vol = volume[None, None, :, :, :].expand(batch_size, -1, -1, -1, -1)
        drr = grid_sample(vol, rays, mode=self.mode, align_corners=align_corners)
        drr = drr[:, 0, 0].sum(dim=-1)
        drr *= raylength / n_points
        return drr

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()